In [8]:
import pandas as pd
import re

In [9]:
train=pd.read_csv("train2.csv")

In [10]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [11]:
def  cleaning_text(df, text_field):
    df[text_field] = df[text_field].str.lower()
    df[text_field] = df[text_field].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    return df

In [12]:
train_clean=cleaning_text(train,"tweet")

In [13]:
train_clean["label"].value_counts()

label
0    29720
1     2242
Name: count, dtype: int64

In [14]:
from sklearn.utils import resample
train_major=train_clean[train_clean.label==0]
train_minor=train_clean[train_clean.label==1]

train_minor_upsampled=resample(train_minor,replace=True,n_samples=len(train_major),
                               random_state=123)
train_upsampled=pd.concat([train_minor_upsampled,train_major])

train_upsampled["label"].value_counts()

label
1    29720
0    29720
Name: count, dtype: int64

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [16]:
pipeline_lg=Pipeline([("vect",CountVectorizer()),
                      ("tfidf",TfidfTransformer()),("lg",LogisticRegression())])

In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(train_upsampled["tweet"],
                                               train_upsampled["label"],random_state=0)

In [18]:
model=pipeline_lg.fit(X_train,y_train)
y_predict=model.predict(X_test)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.99      0.95      0.97      7490
           1       0.95      0.99      0.97      7370

    accuracy                           0.97     14860
   macro avg       0.97      0.97      0.97     14860
weighted avg       0.97      0.97      0.97     14860



In [20]:
pipeline_dt=Pipeline([("vect",CountVectorizer()),
                      ("tfidf",TfidfTransformer()),("dtc",DecisionTreeClassifier())])

In [21]:
model=pipeline_dt.fit(X_train,y_train)
y_predict=model.predict(X_test)

In [22]:
print(classification_report(y_test,y_predict))
      

              precision    recall  f1-score   support

           0       1.00      0.93      0.96      7490
           1       0.93      1.00      0.96      7370

    accuracy                           0.96     14860
   macro avg       0.97      0.96      0.96     14860
weighted avg       0.97      0.96      0.96     14860



In [35]:
pipeline_rf=Pipeline([("vect",CountVectorizer()),
                      ("tfidf",TfidfTransformer()),("rfc",RandomForestClassifier())])

In [39]:
model=pipeline_rf.fit(X_train,y_train)
y_predict=model.predict(X_test)